Before running this notebook, ensure you have the Docker images up and running using the `docker-compose` file provided in this folder. To do so, run the following command in the terminal:

```bash
docker-compose up
```

In a new terminal, run the following command to download the pre-trained model `Phi-3`:

```bash
docker exec -it ollama ollama pull phi3
```

In [1]:
import requests
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/Users/shun_lung_chang/python_projects/llm_zoomcamp/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create a document search engine using Elasticsearch

In [2]:
docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:03<00:00, 288.79it/s]


# Helper functions

In [3]:
def elastic_search(query: str, course: str, es_client: Elasticsearch, index_name: str):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields",
                    }
                },
                "filter": {"term": {"course": course}},
            }
        },
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs


def build_prompt(question: str, response_docs: list[dict[str, str]]) -> str:

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
    """.strip()

    context = ""

    for doc in response_docs:
        context += f"Section: {doc['section']}\n"
        context += f"Question: {doc['question']}\n"
        context += f"Answer: {doc['text']}\n\n"

    return prompt_template.format(question=question, context=context)


def llm(prompt: str, client: OpenAI, model: str = "phi3") -> str:
    response = client.chat.completions.create(
        model=model, messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


def rag(
    query: str,
    course: str,
    model_name: str,
    openai_client: OpenAI,
    es_client: Elasticsearch,
    index_name: str,
) -> str:
    search_results = elastic_search(query, course, es_client, index_name)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, openai_client, model_name)
    return answer

In [4]:
openai_client = OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama",
)

In [5]:
question = "What is the expected duration of the course MLOps zoomcamp?"
course = "mlops-zoomcamp"
model = "phi3"
rag(question, course, model, openai_client, es_client, index_name)

' The expected duration of each module in the MLOps zoomcamp is approximately 3 months, with an additional time frame for completing activities within individual modules such as experiment tracking and capstone projects that can extend up to a total of around 7-8 weeks (including possible extensions). This does not include preparation or other personal study outside of these guidelines.'